<a href="https://colab.research.google.com/github/kalarea/detection-2016-nipsws/blob/master/joint1_slot_gate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[class]对应的分类,然后其他token直接接softmax

In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 1.4MB/s 
     |████████████████████████████████| 1.0MB 6.6MB/s 
     |████████████████████████████████| 870kB 8.9MB/s 
     |████████████████████████████████| 3.7MB 9.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b767231084d78921c2297a7e0e121e974feaf22cb6786722b535207a2b6a32e3
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
#tag和标签管理
BIO_TAG = {
    "检查项目":'INS',
    "科室":'DEP',
    "疾病":'DIS',
    "药品":'DRU',
    "食物":'FOO',
    "症状":'SYM',
    "发病部位":'PAR',
    "治疗方法":'MET',
    "发病人群":'NUM'
}

BIO2ID = {
    'B-INS': 0,
    'I-INS': 1,
    'B-DEP': 2,
    'I-DEP': 3,
    'B-DIS': 4,
    'I-DIS': 5,
    'B-DRU': 6,
    'I-DRU': 7,
    'B-FOO': 8,
    'I-FOO': 9,
    'B-SYM': 10,
    'I-SYM': 11,
    'B-PAR': 12,
    'I-PAR': 13,
    'B-MET': 14,
    'I-MET': 15,
    'B-NUM': 16,
    'I-NUM': 17,
    'O': 18
}

CLASSIFY_TAG = {
    '疾病->并发症':1,
    '疾病->所属科室':2,
    '疾病->药品':3,
    '疾病->食物(宜吃)':4,
    '疾病->食物(忌吃)':5,
    '疾病->症状':6,
    '疾病->简介':7,
    '疾病->发病部位':8,
    '疾病->病因':9,
    '疾病->预防措施':10,
    '疾病->治疗周期':11,
    '疾病->治疗方法':12,
    '疾病->治愈概率':13,
    '疾病->易感人群':14,
    '疾病->检查项目':15,
    '疾病->别名':16,
    '疾病->是否医保':17,
    '疾病->是否传染':18,
    '疾病->费用':19,
    '症状-(疾病)-科室':20,
    '症状-(疾病)-药物':21,
    '症状-(疾病)-食物(宜)':22,
    '症状-(疾病)-食物(忌)':23,
    '症状-(疾病)-检查项目':24,
    '疾病->食物(综合)':25,
    '症状-(疾病)-食物(综合)':26,
    '症状-(疾病)-治疗方法':27,
    '不支持':0
}

ID2BIO = {j:i for i,j in BIO2ID.items()}
ID2CLASS = {j:i for i,j in CLASSIFY_TAG.items()}

In [0]:
#载入数据，
'''
  数据格式为：
  {"id": 2, "text": "一岁半宝宝的食谱",
   "ner_annotations": [{"label": "发病人群", "start_offset": 0, "end_offset": 5}], 
   "classify_annotations": [{"label": "不支持"}]}
'''
data_dict_list = []
import json
with open("/content/drive/My Drive/medical_ner_classification/data/medicalData/data_set.jsonl","r",encoding="utf-8") as f:
  while True:
    line = f.readline()
    if(line ==''): break
    line_json = json.loads(line)
    data_dict_list.append(line_json)

In [0]:
len(data_dict_list)

12045

In [0]:
#对data_dict_list进行切分为train,dev,test
import random

train_rate = 0.8
dev_rate = 0.1
test_rate = 0.1

random.seed(1)
data_len = len(data_dict_list)
print("总共有%d条数据"%(data_len))
random_list = [i for i in range(len(data_dict_list))]
random.shuffle(random_list)

train_indexs = random_list[0:int(train_rate*data_len)]
dev_indexs = random_list[int(train_rate*data_len):int(train_rate*data_len + dev_rate*data_len)]
test_indexs = random_list[int(train_rate*data_len + dev_rate*data_len):]
assert len(train_indexs)+len(dev_indexs)+len(test_indexs)==data_len

总共有12045条数据


In [0]:
train_dict_list = [data_dict_list[i] for i in train_indexs]
dev_dict_list = [data_dict_list[i] for i in dev_indexs]
test_dict_list = [data_dict_list[i] for i in test_indexs]

In [0]:
#构建数据集
from torch.utils.data import Dataset
import random
class Dict_Dataset(Dataset):
  def __init__(self, data_dict_list, tansformer_function, seed = 1):
    self.data_dict_list = data_dict_list
    self.len = len(self.data_dict_list)
    self.transformer = tansformer_function
    self.point = 0    #指向下一个被读取的数据
    random.seed(seed)

  def __getitem__(self,index):
    return self.transformer(self.data_dict_list[index])
  
  def __len__(self):
    return self.len
  
  def set_len(self,len):
    self.len=len
  
  def reset_len(self):
    self.len=len(self.data_dict_list)

  def shuffle(self):
    random.shuffle(self.data_dict_list)
  
  def reset(self):
    self.point = 0
  
  def has_next(self):
    if(self.point>=self.len): return False
    else: return True
  
  def get_batch(self,batch_size):
    num = 0
    input_ids = []
    attention_mask = []
    token_type_ids = []
    loss_mask = []
    ner_tags = []
    ner_labels = []
    class_labels = []
    sequence_lens = [] #用于lstm输入前pack
    #input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_labels
    while(num<batch_size and self.has_next()):
      curr_input_ids, curr_attention_mask, curr_token_type_ids, curr_loss_mask, curr_ner_tags, curr_ner_labels, curr_class_label,\
       curr_sequence_len = self.__getitem__(self.point)
      self.point += 1
      num += 1
      input_ids.append(curr_input_ids)
      attention_mask.append(curr_attention_mask)
      token_type_ids.append(curr_token_type_ids)
      loss_mask.append(curr_loss_mask)
      ner_tags.append(curr_ner_tags)
      ner_labels.append(curr_ner_labels)
      class_labels.append(curr_class_label)
      sequence_lens.append(curr_sequence_len)
    return input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_labels, sequence_lens

In [0]:
#转化器，将字典形式的数据转化为要求的形式
def transformer_dict_to_tensor(max_len,tokenizer,BIO_TAG,BIO2ID):
  '''
    json_dict形如：
    {"id": 2, "text": "一岁半宝宝的食谱",
   "ner_annotations": [{"label": "发病人群", "start_offset": 0, "end_offset": 5}], 
   "classify_annotations": [{"label": "不支持"}]}
  '''
  max_len = max_len
  tokenizer = tokenizer
  BIO_TAG = BIO_TAG
  BIO2ID = BIO2ID

  def transformer(json_dict):
    text = json_dict['text']
    encode = tokenizer.encode_plus(list(text), max_length=max_len, return_token_type_ids=True, return_attention_mask=True, pad_to_max_length=True)
    input_ids = encode['input_ids']     #
    attention_mask = encode['attention_mask']    #
    token_type_ids = encode['token_type_ids']   #
    loss_mask = [0 for i in range(max_len)]   #ner loss mask
    loss_mask[1:1+len(text)] = [1 for i in range(len(text))]
    ner_tags = ['O' for i in range(max_len)] 
    for i in json_dict['ner_annotations']:
      label = i['label']
      start_offset = i['start_offset']
      end_offset = i['end_offset']
      ner_tags[1+start_offset] = 'B-'+BIO_TAG[label]
      for j in range(start_offset+1, end_offset):
        ner_tags[1+j] = 'I-'+BIO_TAG[label]     #
    ner_labels = [BIO2ID[k] for k in ner_tags]    #
    ner_tags = ner_tags[1:1+len(text)]
    class_label = CLASSIFY_TAG[json_dict['classify_annotations'][0]['label']]   #
    sequence_len = len(text)
    return input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len
  return transformer

In [0]:
test_data = {"id": 2, "text": "一岁半宝宝的食谱",
   "ner_annotations": [{"label": "发病人群", "start_offset": 0, "end_offset": 5}], 
   "classify_annotations": [{"label": "不支持"}]}

In [0]:
max_len=81
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
transformer = transformer_dict_to_tensor(max_len,tokenizer,BIO_TAG,BIO2ID)

In [0]:
# input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len = transformer(test_data)

In [0]:
# list(zip(tokenizer.convert_ids_to_tokens(input_ids),attention_mask,token_type_ids,loss_mask,ner_tags, ner_labels))

In [0]:
train_data_set = Dict_Dataset(train_dict_list, transformer, seed = 1)
dev_data_set = Dict_Dataset(dev_dict_list, transformer, seed = 1)
test_data_set = Dict_Dataset(test_dict_list, transformer, seed = 1)

In [0]:
data = train_data_set.get_batch(2)

In [0]:
input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len = data

In [0]:
list(zip(tokenizer.convert_ids_to_tokens(input_ids[0]),attention_mask[0],token_type_ids[0],loss_mask[0],ner_tags[0],ner_labels[0]))

[('[CLS]', 1, 0, 0, 'O', 18),
 ('我', 1, 0, 1, 'O', 18),
 ('患', 1, 0, 1, 'O', 18),
 ('有', 1, 0, 1, 'B-DIS', 18),
 ('松', 1, 0, 1, 'I-DIS', 4),
 ('弛', 1, 0, 1, 'I-DIS', 5),
 ('型', 1, 0, 1, 'I-DIS', 5),
 ('盆', 1, 0, 1, 'I-DIS', 5),
 ('底', 1, 0, 1, 'I-DIS', 5),
 ('功', 1, 0, 1, 'I-DIS', 5),
 ('能', 1, 0, 1, 'I-DIS', 5),
 ('障', 1, 0, 1, 'I-DIS', 5),
 ('碍', 1, 0, 1, 'O', 5),
 ('1', 1, 0, 1, 'O', 18)]

In [0]:
class_label

[0, 9]

In [0]:
sequence_len

[14, 10]

In [0]:
from torch import nn
from transformers import BertModel
from torch.nn import CrossEntropyLoss

In [0]:
class Config(object):
  def __init__(self):
    self.pretrained_bert_name = None
    self.class_label_num = None
    self.ner_tag_num = None
  def describe(self):
    attribute_dict = self.__dict__
    for key, value in attribute_dict.items():
      print(key + " is "+str(value))

In [0]:
class BertJointNerClassify(nn.Module):
  def __init__(self, config):
    super(BertJointNerClassify,self).__init__()
    self.config = config
    self.bert = BertModel.from_pretrained(config.pretrained_bert_name)
    self.classify = nn.Linear(in_features=self.bert.config.hidden_size, out_features=config.class_label_num, bias=True)
    self.emission = nn.Linear(in_features=self.bert.config.hidden_size, out_features=config.ner_tag_num, bias=True)
  def forward(self,input_ids, attention_mask, token_type_ids, loss_mask, ner_labels=None, class_labels=None):
    bert_out, _ = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)#(batch_size,seq_len,bert.config.hidden_size)
    classify_out = self.classify(bert_out[:,0,:]) #(batch_size,1,class_label_num)
    classify_out = classify_out.squeeze(-2) #(batch_size,class_label_num)
    emission_out = self.emission(bert_out)  #(batch_size,seq_len,ner_label_num)
    if ner_labels is not None:
      loss_fct = CrossEntropyLoss(reduction='none')
      ner_loss = loss_fct(emission_out.view(-1,self.config.ner_tag_num),ner_labels.view(-1))
      ner_loss = ner_loss * loss_mask.view(-1)
      ner_output = ner_loss.sum()/input_ids.shape[0]
    else:
      ner_output = emission_out.max(dim=-1).indices.tolist() #(batch_size,seq_len)
    if class_labels is not None:
      loss_fct = CrossEntropyLoss(reduction='sum')
      class_loss = loss_fct(classify_out,class_labels)
      class_output = class_loss/input_ids.shape[0]
    else:
      class_output = classify_out.max(dim=-1).indices.tolist() #(bach_size)
    return ner_output, class_output

In [0]:
config1 = Config()

In [0]:
config1.pretrained_bert_name = "bert-base-chinese"
config1.class_label_num = len(CLASSIFY_TAG)
config1.ner_tag_num = len(BIO2ID)

In [0]:
model = BertJointNerClassify(config1)

In [0]:
# data = train_data_set.get_batch(2)
# input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len = data

In [0]:
# input_ids = torch.tensor(input_ids)
# attention_mask = torch.tensor(attention_mask)
# token_type_ids = torch.tensor(token_type_ids)
# loss_mask = torch.tensor(loss_mask)
# ner_labels = torch.tensor(ner_labels)
# class_label = torch.tensor(class_label)

In [0]:
# input_ids.shape

In [0]:
# a,b = model.forward(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,loss_mask=loss_mask, ner_labels=ner_labels, class_labels=class_label)

In [0]:
# a,b

In [0]:
# len(a[0]),len(a[1])

In [0]:
import torch

In [0]:
#定义优化器
from torch.optim import Adam
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
]
optimizer1 = Adam(optimizer_grouped_parameters, lr=1e-5)

In [0]:
from sklearn.metrics import f1_score
import os
import sys
os.getcwd()
sys.path.append("/content/drive/My Drive/functions")
from get_ner_fmeasure import get_ner_fmeasure

In [0]:
#train
epoch_num = 100
batch_size = 20
model.cuda()
from tqdm import trange
for epoch_i in trange(epoch_num):
  total_loss_ner = 0
  total_loss_class = 0
  model.train()
  train_data_set.shuffle()
  train_data_set.reset()
  num = 0
  while train_data_set.has_next():
    num+=1
    input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len = train_data_set.get_batch(batch_size)
    input_ids = torch.tensor(input_ids).cuda()
    attention_mask = torch.tensor(attention_mask).cuda()
    token_type_ids = torch.tensor(token_type_ids).cuda()
    loss_mask = torch.tensor(loss_mask).cuda()
    ner_labels = torch.tensor(ner_labels).cuda()
    class_label = torch.tensor(class_label).cuda()
    ner_output, class_output = model.forward(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,loss_mask=loss_mask,ner_labels=ner_labels,class_labels=class_label)
    total_loss = ner_output+class_output
    model.zero_grad()
    total_loss.backward()
    optimizer1.step()
    total_loss_ner += ner_output.item()
    total_loss_class += class_output.item()
  total_loss_ner = total_loss_ner/num
  total_loss_class = total_loss_class/num
  #在dev上进行测试
  ner_golds = []
  ner_predicts = []
  class_golds = []
  class_predicts = []
  with torch.no_grad():
    model.eval()
    dev_data_set.reset()
    while dev_data_set.has_next():
      input_ids, attention_mask, token_type_ids, loss_mask, ner_tags, ner_labels, class_label, sequence_len = dev_data_set.get_batch(batch_size=batch_size)
      input_ids = torch.tensor(input_ids).cuda()
      attention_mask = torch.tensor(attention_mask).cuda()
      token_type_ids = torch.tensor(token_type_ids).cuda()
      loss_mask = torch.tensor(loss_mask).cuda()
      ner_labels = torch.tensor(ner_labels).cuda()
      class_label = torch.tensor(class_label).cuda()
      ner_output, class_output = model.forward(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,loss_mask=loss_mask)
      #ner_golds, ner_predicts, class_golds, class_predicts
      ner_gold = ner_tags
      ner_predict = [[ID2BIO[i] for i in ner_output[ii][1:sequence_len[ii]+1]] for ii in range(len(ner_output))]
      class_gold = class_label.tolist()
      class_predict = class_output
      ner_golds += ner_gold
      ner_predicts += ner_predict
      class_golds += class_gold
      class_predicts += class_predict
    #计算ner_f1, class_f1  ner_f1为micro average , class_f1为macro average
    ner_accuracy, ner_precision, ner_recall, ner_f_measure = get_ner_fmeasure(ner_golds, ner_predicts,label_type="BIO")
    class_f1 = f1_score(class_golds,class_predicts,average="micro")
    print('epoch %d , total_loss_ner is %.4f, total_loss_class is %.4f, ner_f_measure is %.4f, class_f1 is %.4f '\
          %(epoch_i, total_loss_ner, total_loss_class, ner_f_measure, class_f1))
    # if(ner_f_measure>ner_f1_base and class_f1>class_f1_base and (ner_f_measure>ner_f1_max or class_f1 >class_f1_max)): 
    #   torch.save(model.state_dict(),model_save_path+"+ner_f1+"+str(ner_f_measure)+"+class_f1+"+str(class_f1)+"+ner_loss+"+str(total_loss_ner)+"+class_loss+"+str(total_loss_class))

  1%|          | 1/100 [01:26<2:22:18, 86.24s/it]

gold_num =  2748  pred_num =  2803  right_num =  1808
epoch 0 , total_loss_ner is 11.4760, total_loss_class is 1.5881, ner_f_measure is 0.6514, class_f1 is 0.6952 


  2%|▏         | 2/100 [02:52<2:20:45, 86.18s/it]

gold_num =  2748  pred_num =  2956  right_num =  1954
epoch 1 , total_loss_ner is 6.4664, total_loss_class is 0.8432, ner_f_measure is 0.6851, class_f1 is 0.7932 


  3%|▎         | 3/100 [04:18<2:19:19, 86.18s/it]

...

gold_num =  2748  pred_num =  2877  right_num =  1948
epoch 38 , total_loss_ner is 0.1834, total_loss_class is 0.0310, ner_f_measure is 0.6926, class_f1 is 0.8306 


 40%|████      | 40/100 [57:34<1:26:23, 86.40s/it]

gold_num =  2748  pred_num =  2798  right_num =  1905
epoch 39 , total_loss_ner is 0.2490, total_loss_class is 0.0360, ner_f_measure is 0.6870, class_f1 is 0.8289 


 41%|████      | 41/100 [59:00<1:24:55, 86.36s/it]

gold_num =  2748  pred_num =  2803  right_num =  1910
epoch 40 , total_loss_ner is 0.1978, total_loss_class is 0.0368, ner_f_measure is 0.6882, class_f1 is 0.8364 


In [0]:
len(dev_data_set)

In [0]:
%debug